# Capstone Project by Niklas

## Problem description & Background

Vehicle accidents can have tremendous impacts on a person's life - both from a health perspective but also on a financial one.
On those, multiple factors like traffic, rain, snow, wind,
or road conditions can determine the likeliness and severity of an accident.

*Can it be possible to determine and quantify the risk even before starting the engine by knowing certain conditions that can be expected during the ride?*

So, the goal of this research is to help people to evaluate the overall risk of a vehicle trip out of the combination of likeliness
of an accident and potential impact/severity, so that drivers can make better decisions for themselves and other people.

## Data description and usage to solve the problem

For this Capstone Project, a Collision data set will be used that includes information about almost 200,000 accidents recorded in Seattle, WA, since the year 2004.
From those accidents, 5085 did not involve vehicles but just pedestrians and cyclists. Those will be excluded from the analysis.
Overall, the data set contains various variables with possible influence on the target variable, the **SEVERITY** of an accident.
These predictor variables include the following:

- *WEATHER:* Describing weather conditions with attribute options like Clear, Overcast, Raining, Fog, Crosswind, Snow, and other
- *ROADCOND:* Describing road conditions with attribute options like Dry, Wet, Ice, Oil, Standing Water, and other
- *LIGHTCOND:* Describing light conditions with attribute options like Daylight, Dawn, and Dark with different status of artificial/street light
- *ADDRTYPE:* Describing the address type where a collision took place with the options Alley, Block, and Intersection
- *COLLISIONTYPE:* Describing the type of the collision with options like parked car, right turn, pedestrian, and others
- *VEHICLECOUNT:* Describing the number of vehicles involved in the accident
- *DATE/TIME:* Describing date and time of when the accident happened

Going forward, this data will be examined via exploratory analysis to identify the most influential predictor variables.


#### Methodology section

1. Exploratory Analysis

In [9]:
#import libraries

import pandas as pd
import numpy as np

In [10]:
# import data

df_raw = pd.read_csv("Data-Collisions.csv")

In [11]:
df_columns = df_raw[['SEVERITYCODE','ADDRTYPE','WEATHER','ROADCOND','LIGHTCOND']]
df_cleaned = df_columns.copy()
df_cleaned.dtypes

SEVERITYCODE     int64
ADDRTYPE        object
WEATHER         object
ROADCOND        object
LIGHTCOND       object
dtype: object

In [12]:
df_cleaned.dropna(inplace=True)

df_cleaned["ADDRTYPE"] = df_cleaned["ADDRTYPE"].astype('category')
df_cleaned["WEATHER"] = df_cleaned["WEATHER"].astype('category')
df_cleaned["ROADCOND"] = df_cleaned["ROADCOND"].astype('category')
df_cleaned["LIGHTCOND"] = df_cleaned["LIGHTCOND"].astype('category')

df_cleaned["ADDRTYPE"].value_counts()

df_cleaned["WEATHER"].value_counts()

df_cleaned["ROADCOND"].value_counts()

df_cleaned["LIGHTCOND"].value_counts()

df_cleaned["ADDRTYPE_INT"] = df_cleaned["ADDRTYPE"].cat.codes
df_cleaned["WEATHER_INT"] = df_cleaned["WEATHER"].cat.codes
df_cleaned["ROADCOND_INT"] = df_cleaned["ROADCOND"].cat.codes
df_cleaned["LIGHTCOND_INT"] = df_cleaned["LIGHTCOND"].cat.codes

2. Inferential statistical testing

3. Machine Learnings

#### Results

#### Discussion

#### Conclusion